# **Python - Original Code**

In [54]:
import numpy as np
import pandas as pd

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [55]:
def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

In [56]:
def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i, x_name] not in value_dict.keys():
      value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] = 1
    else:
      value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result

In [57]:
%%timeit
result_1 = target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 27.1 s per loop


In [66]:
%%timeit
result_2 = target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 281 ms per loop


In [59]:
diff = np.linalg.norm(result_1 - result_2)
print(diff)

0.0


# **Cython-Improved Code**

In [ ]:
%load_ext Cython

In [74]:
%%cython

import cython
import pandas as pd
import numpy as np
cimport numpy as cnp

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] target_mean_c1(cnp.ndarray[int,ndim=2] data):
  cdef cnp.ndarray[int] x = data[:,1]
  cdef cnp.ndarray[int] y = data[:,0]
  cdef int data_shape = data.shape[0]
  cdef cnp.ndarray[double] result = np.zeros(data_shape)
  cdef cnp.ndarray[int] dict_value = np.zeros(data_shape).astype(np.intc)
  cdef cnp.ndarray[int] dict_count = np.zeros(data_shape).astype(np.intc)
  cdef int i
  for i in range(data_shape):
    dict_value[x[i]] += y[i]
    dict_count[x[i]] += 1
  for i in range(data_shape):
    result[i] = (dict_value[x[i]] - y[i]) / (dict_count[x[i]] - 1)
  return result

In [76]:
%%timeit
result_c1 = target_mean_c1(data.values.astype(np.intc))

The slowest run took 16.45 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 42.7 µs per loop
